In [200]:
import numpy as np
import pandas as pd
import nltk
import re
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
from collections import Counter
import operator

In [302]:
df = pd.read_csv('webmd1.csv')

# Data Manipulation

In [270]:
otc = df[df.Type == ' otc']
cols = ['Type','Indication','Condition','Drug','BrandName','GenName','Use','HowtoUse','Precautions','AvoidUse',\
              'Allergies','Interactions','Sides','Effectiveness','EaseofUse','Satisfaction',\
              'Review','Form','PkgCount','EstimatedPrice']
otc = otc[cols]
otc.rename(columns = {'Review':'Reviews'}, inplace=True) # rename to Reviews
otc.reset_index(drop=True, inplace=True)

In [271]:
ind = otc.loc[(otc.Reviews == 'Be the first to review it') &\
        ((otc.Effectiveness != ' ') | (otc.EaseofUse != ' ') | (otc.Satisfaction != ' '))].index
otc.loc[ind,'Reviews'] = 1 # fix rows with ratings but no reviews

ind2 = otc.loc[otc.Reviews == 'Be the first to review it'].index
otc.loc[ind2, 'Reviews'] = 0 # change rows with no reviews to value 0

ind3 = otc.loc[(otc.Reviews != 0)&(otc.Reviews.str.contains('User Reviews'))].index
otc.loc[ind3, 'Reviews'] = otc.loc[ind3].Reviews.str.replace('User Reviews', '') # remove text from reviews

otc.Reviews = pd.to_numeric(otc.Reviews) # convert to numeric

In [273]:
otc.loc[otc.Reviews == 0,['Effectiveness','EaseofUse','Satisfaction']] = np.nan

In [275]:
ind4 = otc.loc[otc.BrandName == ' '].index
otc.loc[ind4,'BrandName'] = np.nan
ind5 = otc.loc[otc.GenName == ' '].index
otc.loc[ind5, 'GenName'] = np.nan

otc.loc[otc.loc[otc.Use == ' '].index, 'Use'] = np.nan
otc.loc[otc.loc[otc.HowtoUse == ' '].index, 'HowtoUse'] = np.nan
otc.loc[otc.loc[otc.Precautions == ' '].index, 'Precautions'] = np.nan
otc.loc[otc.loc[otc.Interactions == ' '].index, 'Interactions'] = np.nan
otc.loc[otc.loc[otc.Sides == ' '].index, 'Sides'] = np.nan

In [276]:
otc.loc[otc.loc[otc.Allergies == ' '].index, 'Allergies'] = np.nan
otc.loc[otc.loc[otc.AvoidUse == ' '].index, 'AvoidUse'] = np.nan
otc.loc[otc.loc[(otc.Form == ' ') & (otc.PkgCount == ' ') & (otc.EstimatedPrice == ' ')].index,\
       ['Form','PkgCount','EstimatedPrice']] = np.nan

In [277]:
otc.sort_values('Reviews',ascending=False)

,Type,Indication,Condition,Drug,BrandName,GenName,Use,HowtoUse,Precautions,AvoidUse,Allergies,Interactions,Sides,Effectiveness,EaseofUse,Satisfaction,Reviews,Form,PkgCount,EstimatedPrice
17566,otc,label,Cough,Mucinex oral,NaN,GUAIFENESIN,This medication is used for the temporary re...,Take this medication by mouth with or withou...,"Before taking guaifenesin, tell your doctor or...",NaN,GUAIFENESIN,To help your doctor and pharmacist give you th...,Nausea or vomiting may occur. If either of ...,3.08656,3.797266,2.583143,439,tab,20,15.822
17642,otc,label,Cold Symptoms,Mucinex oral,NaN,GUAIFENESIN,This medication is used for the temporary re...,Take this medication by mouth with or withou...,"Before taking guaifenesin, tell your doctor or...",NaN,GUAIFENESIN,To help your doctor and pharmacist give you th...,Nausea or vomiting may occur. If either of ...,3.08656,3.797266,2.583143,439,tab,20,15.822
17555,otc,label,Cough,Mucinex DM oral,NaN,GUAIFENESIN/DEXTROMETHORPHAN HBR,This combination medication is used to relie...,Take this medication by mouth with or withou...,"Before taking this drug, tell your doctor or p...",systemic mastocytosis,DEXTROMETHORPHAN ANALOGU GUAIFENESIN,Taking certain MAO inhibitors with this medica...,"Dizziness , drowsiness, nausea , and vomitin...",3.321608,3.743719,2.492462,398,syrup,240,10.969
17640,otc,label,Cold Symptoms,Mucinex DM oral,NaN,GUAIFENESIN/DEXTROMETHORPHAN HBR,This combination medication is used to relie...,Take this medication by mouth with or withou...,"Before taking this drug, tell your doctor or p...",systemic mastocytosis,DEXTROMETHORPHAN ANALOGU GUAIFENESIN,Taking certain MAO inhibitors with this medica...,"Dizziness , drowsiness, nausea , and vomitin...",3.321608,3.743719,2.492462,398,syrup,240,10.969
3824,otc,label,Incomplete or Infrequent Bowel Movements,Dulcolax (bisacodyl) oral,NaN,BISACODYL,Bisacodyl is used to treat constipation . It...,Take this medication by mouth as directed ...,"Before taking bisacodyl, tell your doctor or p...","Appendicitis, Stomach or Intestine Blockage, U...",BISACODYL,The effects of some drugs can change if you ta...,"Stomach / abdominal pain or cramping, nausea...",3.85283,3.260377,2.418868,265,tablet,4,7.3506
19379,otc,label,Emptying of the Bowel,Dulcolax (bisacodyl) oral,NaN,BISACODYL,Bisacodyl is used to treat constipation . It...,Take this medication by mouth as directed ...,"Before taking bisacodyl, tell your doctor or p...","Appendicitis, Stomach or Intestine Blockage, U...",BISACODYL,The effects of some drugs can change if you ta...,"Stomach / abdominal pain or cramping, nausea...",3.85283,3.260377,2.418868,265,tablet,4,7.3506
3823,otc,label,Incomplete or Infrequent Bowel Movements,Miralax oral,NaN,POLYETHYLENE GLYCOL 3350,This medication is used to treat occasional ...,"Take by mouth usually once daily, or as dire...","Before taking this medication, tell your docto...","Problem Emptying Stomach Contents, Severe Ulce...",POLYETHYLENE GLYCOL ANALOGUES,"Before using this medication, tell your doctor...","Nausea , abdominal cramping, or gas may occur....",3.764706,4.230769,3.248869,221,powd,14,32.35
2053,otc,label,Parkinson's Disease,Benadryl oral,NaN,DIPHENHYDRAMINE HCL,Diphenhydramine is an antihistamine used to...,Follow all directions on the product package. ...,"Before taking diphenhydramine, tell your docto...","Increased Pressure in the Eye, closed angle gl...",ANTIHISTAMINES ANTIHISTAMINES - ETHANOLAMINE,Some products that may interact with this drug...,"Drowsiness, dizziness, constipation , stomac...",3.794595,4.291892,3.497297,185,capsule,30,8.1898
4857,otc,label,Motion Sickness,Benadryl oral,NaN,DIPHENHYDRAMINE HCL,Diphenhydramine is an antihistamine used to...,Follow all directions on the product package. ...,"Before taking diphenhydramine, tell your docto...","Increased Pressure in the Eye, closed angle gl...",ANTIHISTAMINES ANTIHISTAMINES - ETHANOLAMINE,Some products that may interact with this drug...,"Drowsiness, dizziness, constip

In [303]:
otc.Satisfaction = pd.to_numeric(otc.Satisfaction)
otc.EaseofUse = pd.to_numeric(otc.EaseofUse)
otc.Effectiveness = pd.to_numeric(otc.Effectiveness)
otc.EstimatedPrice = pd.to_numeric(otc.EstimatedPrice)

# Data Visualization

In [296]:
colFun = {'Drug':['size'],'Reviews':['max','median'],\
                                'EaseofUse':['max','median'],'Satisfaction':['max','median'],\
                               'Effectiveness':['max','median'],'EstimatedPrice':['mean']}

otc.groupby('Condition').agg(len).sort_values('Reviews',ascending=False)
group = otc.groupby(['Condition']).agg(colFun)
groupcols = ['Drug','Effectiveness','EaseofUse','Satisfaction','Reviews','EstimatedPrice']
# group.loc[group.Drug.sort_values('size',ascending=False).index][groupcols]

# Robo-Pharamist Drug Finder

In [325]:
search = ['Indication','Condition','Drug',\
              'Effectiveness','EaseofUse','Satisfaction',\
              'Reviews','Form','PkgCount','EstimatedPrice']
#['Use','HowtoUse','Precautions','AvoidUse','Allergies','Interactions','Sides''BrandName','GenName',] # columns removed
def search_conditions(string, tg2, x, dd1, tg1, na, form, count):
    index = []
    df = otc
    if na:
        df = df.loc[(df.EaseofUse >= 0)&(df.EstimatedPrice >= 0)].reset_index(drop=True)
        
    if form != 'All':
        df = df.loc[df.Form == form].reset_index(drop=True)
                
    if tg2:
        for i in range(len(df)):
            if re.search('^'+string.lower()+'$', df.Condition[i].lower()):
                index += [i]
            else:
                continue
        return df.loc[index].sort_values(dd1, ascending=tg1).head(count).set_index('Drug')
    
    else:
        for i in range(len(df)):
            if re.search(string.lower(), df.Condition[i].lower()):
                index += [i]
            else:
                continue
        return df[search].loc[index].sort_values(dd1, ascending=tg1).head(count).set_index('Drug') 
    
search_conditions('cough',False,'','Effectiveness',False,True,'All',5)

,Indication,Condition,Effectiveness,EaseofUse,Satisfaction,Reviews,Form,PkgCount,EstimatedPrice
Drug,,,,,,,,,
Tussin Expectorant oral,label,Cough,5.0,5.0,5.0,1,tab,20,15.8220
J-MAX oral,label,Cough,5.0,5.0,5.0,1,tablet,20,19.9769
Mar-Cof CG oral,label,Cough,5.0,5.0,5.0,1,liquid,120,10.8479
Poly-Vent IR oral,label,Cough,5.0,5.0,5.0,1,tab,18,17.1630
Aler-Cap oral,label,Cough,5.0,4.0,1.0,1,capsule,30,8.1898


### Widget Creation

In [300]:
tg1 = widgets.ToggleButton(
    value=False,
    description='Ascending/Descending',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

tg2 = widgets.ToggleButton(
    value=False,
    description='Exact Match',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

dd1 = widgets.Dropdown(
    options={'Estimated Price': 'EstimatedPrice', 'Reviews': 'Reviews', 'Ease of Use': 'EaseofUse',\
            'Satisfaction': 'Satisfaction', 'Effectiveness': 'Effectiveness'},
    value='Reviews',
    description='Sort by:',
    disabled=False,
    button_style='info' # 'success', 'info', 'warning', 'danger' or ''
)

str1 = widgets.Text(
    value='Cough',
    placeholder='',
    description='Search:',
    disabled=False
)

int1 = widgets.IntSlider(
    value = 5,
    step = 1,
    max = 10,
    description = 'Show:'
)

na1 = widgets.ToggleButton(
    value=True,
    description='Remove NA',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

for1 = widgets.Dropdown(
    options=['All']+['tab'],
    value='All',
    description='Form:',
    disabled=False,
    button_style='info' # 'success', 'info', 'warning', 'danger' or ''
)

In [330]:
interactive(search_conditions,
            string = str1,
            tg2 = tg2,
            x = '',
            dd1 = dd1,
            tg1 = tg1,
            na = na1,
            form = for1,
            count = int1,
            __manual=False)

,Indication,Condition,Effectiveness,EaseofUse,Satisfaction,Reviews,Form,PkgCount,EstimatedPrice
Drug,,,,,,,,,
Cold and Allergy PE oral,label,Cold Symptoms,5.0,5.0,5.0,1,tablet,30,11.4700
Aquanaz oral,label,Cold Symptoms,5.0,5.0,5.0,1,tablet,30,26.9236
CapCof oral,label,Cold Symptoms,5.0,5.0,5.0,1,liquid,60,11.5447
Poly-Vent IR oral,label,Cold Symptoms,5.0,5.0,5.0,1,tab,18,17.1630
Ru-Hist D oral,label,Cold Symptoms,5.0,5.0,5.0,1,tablet,20,22.2269


# NLTK

In [19]:
strings = []
for i in range(len(otc.Drug)):
    strings += nltk.word_tokenize(otc.Drug[i])

In [22]:
stringscount = Counter(strings)
sorted(stringscount.items(), key=operator.itemgetter(1), reverse=True)

[('oral', 15954),
 ('topical', 2348),
 ('(', 2151),
 (')', 2151),
 ('Allergy', 1418),
 ('Relief', 1385),
 ("'s", 1040),
 ('and', 1018),
 ('Cold', 913),
 ('Children', 797),
 ('Pain', 596),
 ('Strength', 534),
 ('PE', 520),
 ('Plus', 501),
 ('DM', 465),
 ('Cough', 444),
 ('Sinus', 416),
 ('Aspirin', 393),
 ('D', 385),
 ('with', 339),
 ('Sleep', 327),
 ('D3', 318),
 ('Severe', 298),
 ('Extra', 272),
 ('nasal', 271),
 ('PM', 268),
 ('Calcium', 267),
 ('Antacid', 249),
 ('Cold-Flu', 244),
 ('diphenhydramine', 242),
 ('calcium', 240),
 ('Vitamin', 209),
 ('Nighttime', 208),
 ('Time', 206),
 ('rectal', 203),
 ('Multi-Symptom', 202),
 ('Complete', 199),
 ('Night', 198),
 ('Child', 197),
 ('Daily', 195),
 ('Flu', 184),
 ('Congestion', 182),
 ('Reliever', 175),
 ('Day', 173),
 ('Vicks', 163),
 ('+', 159),
 ('vitamin', 158),
 ('Mucinex', 158),
 ('ophthalmic', 154),
 ('Bayer', 152),
 ('Max', 146),
 ('Formula', 144),
 ('Anti-Itch', 138),
 ('Alka-Seltzer', 137),
 ('Advil', 133),
 ('Cold-Cough', 132)